# Object Detection by using Tensorflow and OpenCv

***
**Introduction**
***

This notebook takes you step by step to create object detection model by using Tensorflow Object Detection API's <br><br>

**Enviroment Specifications**:<br>
- Windows 10 Enterpise
- python version(3.8.8)
- TensorFlow version(2.4.1) (Facing issues with 2.4.2)
- numpy version version 1.19.5<br><br>


**prerequisites** 
- Installing Tensorflow object Detections API's you can refer to below link [Installation Guide](http://https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html)<br>
Note: As part of this installation many other depdencies need to be installed as well, please make sure to go through all installation steps, theyare bit long and may you would face some challenges

***
**Imports**
***

In [2]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import cv2 
import numpy as np

***

**Defining the Requiered Paths and Directories**

***

In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPT_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = 'Tensorflow/workspace/annotations'
IMAGE_PATH = 'Tensorflow/workspace/images'
MODEL_PATH = 'Tensorflow/workspace/models'
PRETRAINED_MODEL_PATH = 'Tensorflow/workspace/pre-trained-models'
CONFIG_PATH = MODEL_PATH + '/my_sdd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH + '/my_sdd_mobnet//'


***
**Label Mapp**
***
Before running the below code you need to open the label the image by running the labelImag.py file

In [2]:
labels = [{'name':'can', 'id':1},{'name':'glass', 'id':2},{'name':'plastic', 'id':3}]

with open(ANNOTATION_PATH + '/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item{ \n')
        f.write('\tid:{}\n'.format(label['id']))
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('}\n')

***
**Creating Tfrecord files**

***

In [3]:
!python {SCRIPT_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPT_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: RealTimeObjectDetection/Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: RealTimeObjectDetection/Tensorflow/workspace/annotations/test.record


***
**Creating model directory and copy the pipeline config file to the model directory**
***

In [5]:
CUSTOM_MODEL_NAME = 'my_sdd_mobnet'
!mkdir {'RealTimeObjectDetection\Tensorflow\workspace\models\\' + CUSTOM_MODEL_NAME}
!copy RealTimeObjectDetection\Tensorflow\workspace\pre-trained-models\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\pipeline.config RealTimeObjectDetection\Tensorflow\workspace\models\my_sdd_mobnet\pipeline.config

A subdirectory or file RealTimeObjectDetection\Tensorflow\workspace\models\my_sdd_mobnet already exists.


        1 file(s) copied.


***
**Configure the pipeline file**
***

In [4]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()

In [5]:
with tf.io.gfile.GFile(CONFIG_PATH, 'r') as f:
    proto_str = f.read()
    text_format.Merge(proto_str,pipeline_config)
    

In [6]:
pipeline_config.model.ssd.num_classes = 3
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [7]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text) 

***

**Running training from notebook**

***
Note: you need to replace the path of model_main_tf2.py with your path

In [ ]:
!python C:\Users\Windows10-Desktop\Tensorflow\models\research\object_detection/model_main_tf2.py --model_dir=RealTimeObjectDetection/Tensorflow/workspace/models/my_sdd_mobnet --pipeline_config_path=C:\Users\Windows10-Desktop\RealTimeObjectDetection\Tensorflow\workspace\models\my_sdd_mobnet\pipeline.config --num_train_steps=5000

***
**Running training from Anaconda Prompt**

***
Note: you need to replace the path of model_main_tf2.py with your path

In [ ]:
!python C:\Users\Windows10-Desktop\Tensorflow\models\research\object_detection/model_main_tf2.py --model_dir=RealTimeObjectDetection/Tensorflow/workspace/models/my_sdd_mobnet --pipeline_config_path=C:\Users\Windows10-Desktop\RealTimeObjectDetection\Tensorflow\workspace\models\my_sdd_mobnet\pipeline.config --num_train_steps=5000

***
**Inference**
***

In [10]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-8')).expect_partial()

In [11]:
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [13]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [32]:
### Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('test24.mp4',fourcc, 20.0, (width,height),isColor=True)

In [ ]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.7,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (1000, 600)))
    out.write(image_np_with_detections)
    
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        break